In [1]:
using StateSpaceDynamics
using Random
using Distributions
using Plots
using LinearAlgebra
using ForwardDiff
using StatsFuns
using Optim

In [2]:
# generate synthetic data for a switching Gaussian Regression
Random.seed!(12345)

n = 1000

beta_1 = [1.0, 2.0]
beta_2 = [3.0, 4.0]

sigma_1 = 0.5
sigma_2 = 0.5

# Simualte data from a switching GLM
A = [0.9 0.1; 0.1 0.9]
pi_k = [0.5, 0.5]

X = randn(n, 2)
Y = zeros(n)
states = zeros(Int, n)

# First state
states[1] = Int(rand() < pi_k[1] ? 1 : 2)
Y[1] = if states[1] == 1
    beta_1[1] + beta_1[2] * X[1, 1] + sigma_1 * randn()
else
    beta_2[1] + beta_2[2] * X[1, 1] + sigma_2 * randn()
end

# Remaining states
for i in 2:n
    states[i] = Int(rand() < A[states[i-1], 1] ? 1 : 2)
    Y[i] = if states[i] == 1
        beta_1[1] + beta_1[2] * X[i, 1] + sigma_1 * randn()
    else
        beta_2[1] + beta_2[2] * X[i, 1] + sigma_2 * randn()
    end
end


In [3]:
# reshape x and y to be matrices
X_input = reshape(X[:, 1], n, 1)
Y = reshape(Y, n, 1)

est_model = SwitchingGaussianRegression(K=2, input_dim=1, output_dim=1)
fit!(est_model, X_input, Y)

Running EM algorithm...   2%|█                           |  ETA: 0:05:38

Running EM algorithm... 100%|████████████████████████████| Time: 0:00:08


5-element Vector{Float64}:
  -Inf
 -514.2341165866646
 -104.12836956661128
 -104.12836658096036
 -104.1283663486642

In [4]:
# generate synthetic data
Random.seed!(123)

# set parameters
n = 1000
β = [0.1, 0.9]

# generate data
X = hcat(ones(n), randn(n, 1))

p = logistic.(X * β)
y = rand.(Bernoulli.(p));

logreg = SSM.BernoulliRegression()
SSM.fit!(logreg, reshape(X[:, 2], length(y), 1), y)

UndefVarError: UndefVarError: `SSM` not defined

In [5]:
model = SSM.SwitchingBernoulliRegression(;K=2, λ=0.1)
SSM.fit!(model, reshape(X[:, 2], length(y), 1), y, 10)

UndefVarError: UndefVarError: `SSM` not defined

In [6]:
# synthetic poisson regression data
Random.seed!(123)

# set parameters
n = 100
β = [0.1, 0.9]
# generate data

X = hcat(ones(n), randn(n, 1))
λ = exp.(X * β)

y = rand.(Poisson.(λ))

poisreg = SSM.PoissonRegression()
SSM.fit!(poisreg, reshape(X[:, 2], length(y), 1), y)

UndefVarError: UndefVarError: `SSM` not defined

In [7]:
model = SSM.SwitchingPoissonRegression(;K=2, λ=0.1)
fit!(model, reshape(X[:, 2], length(y), 1), y)

UndefVarError: UndefVarError: `SSM` not defined

In [8]:
# Parameters
n = 1000
β₁ = [0.1, 0.9]
β₂ = [0.9, 0.1]
σ²₁ = 0.1
σ²₂ = 0.1
A = [0.9 0.1; 0.1 0.9]

# Data generation
X = hcat(ones(n), randn(n, 1))
y = zeros(n)
state = Vector{Int}(undef, n)

for i in 1:n
    if i == 1
        # choose a state at random
        state[i] = rand(1:2)
    else
        # Transition based on previous state
        state[i] = rand(Categorical(A[state[i-1], :]))
    end

    if state[i] == 1
        y[i] = X[i, :]' * β₁ + sqrt(σ²₁) * randn()
    else
        y[i] = X[i, :]' * β₂ + sqrt(σ²₂) * randn()
    end
end

# Fit linear regression model
linreg = SSM.SwitchingGaussianRegression(; K=2, λ=0.1)
SSM.fit!(linreg, reshape(X[:, 2], length(y), 1), y, 10000, 1e-6)

println("Fitted coefficients: ", linreg.B[1].regression.β)


UndefVarError: UndefVarError: `SSM` not defined

In [9]:
γ, ξ, α, β = SSM.E_step(linreg, reshape(X[:, 2], length(y), 1), y)

UndefVarError: UndefVarError: `SSM` not defined